Here I am just setting up the DataFrame same as in Parts 1 & 2.

In [1]:
import requests
import re
import pandas as pd
from io import StringIO

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_tor_wikipedia_page = requests.get(wikipedia_link)
page = raw_tor_wikipedia_page.text
postal = page[(page.find('<tbody>')+len('<tbody')):page.find('</tbody>')]

p0 =re.sub('<.*?>','|',postal)
p1 =str.replace(p0,'||','|')
p2 =str.replace(p1,'\n|\n|\n|','\n')
p3 =str.replace(p2,'|\n|','|')
p4 =str.replace(p3,'|\n','\n').strip('>|')

data = StringIO(p4)
df = pd.read_csv(data, sep='|')
df = df[df.Borough != 'Not assigned']

df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: '%s' % ', '.join(x))
df = pd.DataFrame(df).reset_index()

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

latlong_df = pd.read_csv('http://cocl.us/Geospatial_data')

final_df = df.merge(latlong_df, left_on='Postcode', right_on='Postal Code')
final_df.drop('Postal Code',axis=1, inplace=True)
final_df.rename(columns={'Postcode':'PostalCode'}, inplace=True)
final_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Importing required libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


Get lat and long for Toronto and print it; then create the map with the neighborhoods in it.

In [3]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


In [4]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare credentials

In [5]:
CLIENT_ID = 'XXXXXXXXXX'
CLIENT_SECRET = 'XXXXXXXX'
VERSION = '20190227'
LIMIT = 1
radius = 500

Explore all neighbourhoods in Toronto. What I want to know here is just the top venue in each postal code/neighbourhoods.

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Running this function for every neighbourhood group.

In [7]:
top_toronto_venues = getNearbyVenues(names=final_df['Neighbourhood'],
                                   latitudes=final_df['Latitude'],
                                   longitudes=final_df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [8]:
#see what the output looks like
print(top_toronto_venues.shape)
top_toronto_venues.head()

(100, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
4,Cedarbrae,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant


One hot encode so we can run k means on this.

In [9]:
# one hot encoding
toronto_onehot = pd.get_dummies(top_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = top_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[46]] + list(toronto_onehot.columns[0:45])+ list(toronto_onehot.columns[47:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighborhood,Airport,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,...,Pharmacy,Pizza Place,Playground,Pool,Restaurant,Sandwich Place,Skating Rink,Sports Bar,Toy / Game Store,Trail
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Scarborough Village,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,"East Birchmount Park, Ionview, Kennedy Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Clairlea, Golden Mile, Oakridge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Cliffcrest, Cliffside, Scarborough Village West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I did 8 clusters here. Curious as to ow these will be grouped using just a # 1 venue.

In [10]:
# set number of clusters
kclusters = 8

toronto_onehot2 = toronto_onehot.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_onehot2)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 1, 5, 0, 0, 0, 0, 0, 3], dtype=int32)

Add clustering labels to a new df for use in the map; then create the map

In [11]:
top_tor_2 = top_toronto_venues

top_tor_2.insert(0, 'Cluster Labels', kmeans.labels_)
top_tor_2.head()


,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,1,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,5,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
4,0,Cedarbrae,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant


In [12]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(top_tor_2['Neighborhood Latitude'], top_tor_2['Neighborhood Longitude'], top_tor_2['Neighborhood'], top_tor_2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the clusters.  It looks like Cluster 0 is a miscellaneous (one off restaurants and such) while places with destinations like parks, or bars, or grocery stores show up in other clusters.

To find closer similarities, we could copy the NYC lab, but I wanted to see what a single data point would generate.  Fun fact, looks like the most sought after pizza and grocery stores are on the edge of the city.

In [17]:
top_tor_2.loc[top_tor_2['Cluster Labels'] == 7, top_tor_2.columns[[1] + list(range(5, top_tor_2.shape[1]))]]

,Neighborhood,Venue Latitude,Venue Longitude,Venue Category
20,Willowdale South,43.768648,-79.412597,Grocery Store
73,Christie,43.668471,-79.420485,Grocery Store
98,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.741968,-79.586639,Grocery Store
